<a href="https://colab.research.google.com/github/JoshStrother/DataVisualization/blob/main/lesson7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


Our question is: What type of subscription is the most popular? Any thoughts as to why this might be? The second part to this is how much time in years, months,days, hours and minutes total have been completed by this subscriber type?

The first thing we will do is get the table that has our subscriber types on it.

In [10]:
%%bigquery --project data-viz-338716

SELECT*
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
LIMIT 10

,trip_id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,9900289692,Walk Up,248,2015-10-02 21:12:01+00:00,1006,Zilker Park West,1008,Nueces @ 3rd,39
1,9900285987,24-Hour Kiosk (Austin B-cycle),446,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
2,9900285989,24-Hour Kiosk (Austin B-cycle),203,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
3,9900285991,24-Hour Kiosk (Austin B-cycle),101,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,30
4,9900286140,24-Hour Kiosk (Austin B-cycle),242,2014-10-26 18:12:00+00:00,2541,State Capitol @ 14th & Colorado,2541,State Capitol @ 14th & Colorado,19
5,9900286143,24-Hour Kiosk (Austin B-cycle),924,2014-10-26 18:12:00+00:00,2541,State Capitol @ 14th & Colorado,2541,State Capitol @ 14th & Colorado,17
6,9900286171,24-Hour Kiosk (Austin B-cycle),869,2014-10-26 18:12:00+00:00,2536,Waller & 6th St.,2536,Waller & 6th St.,6
7,9900286214,Annual Membership (Austin B-cycle),24,2014-10-26 20:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,0
8,9900286540,24-Hour Kiosk (Austin B-cycle),117,2014-10-27 15:12:00+00:00,2536,Waller & 6th St.,2536,Waller & 6th St.,12
9,13575843,Walk Up,302,2017-01-29 16:42:52+00:00,3464,Pease Park,3464,Pease Park,47


Okay now we will bring out only the sub type and count how many times it appears.  We will also order this by DESC in order to get greaters amount of subs to least.

In [7]:
%%bigquery --project data-viz-338716

SELECT subscriber_type, COUNT(subscriber_type) as num_of_sub
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
GROUP BY subscriber_type
ORDER BY num_of_sub DESC

,subscriber_type,num_of_sub
0,Walk Up,374151
1,U.T. Student Membership,289288
2,Local365,260376
3,24-Hour Kiosk (Austin B-cycle),108672
4,Local30,69062
...,...,...
72,Annual Pass (30 minute),2
73,24-Hour Membership (Austin B-cycle),1
74,Heartland Pass (Annual Pay),1
75,Heartland Pass (Monthly Pay),1


After viewing the data is looks like walk ups are the most common kind of subscirber.  Which makes since because sometimes you may need to borrow a bike just that one wether it be that your car is broke down or you just want a nature ride out of the house.  

Next we need to gather just the walk up duration sum and then convert that into years, months, days, hours, minutes.  This is the hard part... so first of all im going to just see how many minutes in total there are.

In [13]:
%%bigquery --project data-viz-338716

SELECT SUM(duration_minutes) as total_time
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
WHERE subscriber_type = 'Walk Up'

,total_time
0,15928365


So obviously I can just divide it out separately and then tell you the years, month... but I want to try and do it in the query.  First im going to try to just calculate the years.

In [18]:
%%bigquery --project data-viz-338716
SELECT total_time/60/24/365 as years
FROM (SELECT SUM(duration_minutes) as total_time
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
WHERE subscriber_type = 'Walk Up')

,years
0,30.305108


Okay so that does in fact check out, now we need to add the FLOOR() function in order to show just the actual full years. Then from there we will see if we can't get the months ... minutes.

In [21]:
%%bigquery --project data-viz-338716
SELECT FLOOR(total_time/60/24/365) as years, FLOOR(MOD(total_time/60/24/365)*12) as months
FROM (SELECT SUM(duration_minutes) as total_time
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
WHERE subscriber_type = 'Walk Up')

Executing query with job ID: ce0daf34-2ed0-40b1-8344-b32929c834e6
Query executing: 0.25s


ERROR:
 400 No matching signature for function MOD for argument types: INT64. Supported signatures: MOD(INT64, INT64); MOD(NUMERIC, NUMERIC); MOD(BIGNUMERIC, BIGNUMERIC) at [1:46]

(job ID: ce0daf34-2ed0-40b1-8344-b32929c834e6)

                      -----Query Job SQL Follows-----                      

    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:SELECT FLOOR(total_time/60/24/365) as years, MOD(total_time) as months
   2:FROM (SELECT SUM(duration_minutes) as total_time
   3:FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
   4:WHERE subscriber_type = 'Walk Up')
    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |


Alright.....So I wanted grab the remainder off the years by using MOD() but apparently it doesn't like that..not sure where to go next at the moment.

Okay so after looking closer it looks like I'm using the MOD() function wrong! it is supposed to be MOD(x, y) where x is divided by y. Let's try this again.

In [23]:
%%bigquery --project data-viz-338716
SELECT FLOOR(total_time/60/24/365) as years, FLOOR(MOD(total_time, (60/24/365))*12) as months
FROM (SELECT SUM(duration_minutes) as total_time
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
WHERE subscriber_type = 'Walk Up')

Executing query with job ID: 7117e573-afa6-4762-aa5e-d8a71393259c
Query executing: 0.27s


ERROR:
 400 No matching signature for function MOD for argument types: INT64, FLOAT64. Supported signatures: MOD(INT64, INT64); MOD(NUMERIC, NUMERIC); MOD(BIGNUMERIC, BIGNUMERIC) at [1:52]

(job ID: 7117e573-afa6-4762-aa5e-d8a71393259c)

                                 -----Query Job SQL Follows-----                                  

    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:SELECT FLOOR(total_time/60/24/365) as years, FLOOR(MOD(total_time, (60/24/365))*12) as months
   2:FROM (SELECT SUM(duration_minutes) as total_time
   3:FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
   4:WHERE subscriber_type = 'Walk Up')
    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |


So the second part im trying to use is a float..not really sure where to continue but I did see that there is a datetime function (might not be a function but it exists).  Im not sure of how to use it but we will see if maybe we can use that to convert the minutes? probably not.

Upon even further inspection, date time is not what we need because it reads it as an actual calendar date rather than an amount of time.

I have worked on this over an hour and a half in total and can not seem to figure out how to either A. grab just the decimal portion of a number to work with or B. find a function that does the work for me.  I have officially been stumped.